In [ ]:
import torch
import torch.nn.functional as F
from torch.distributions import Categorical
import torch.nn as nn
import numpy as np
import tqdm
import time
import pickle, os

CUDA_device = 0
eps = 1e-12

def soft_cross_entropy(
    pred, target_indices, target_probas, weight=None, reduction="mean", ignore_index=-1
):
    """Computes the cross entropy loss using soft labels.

    Here the soft labels are defined through the parameters `target_indices`
    and `target_probas`. They respectively represent the class indices involved
    in the target distribution and their corresponding probability.
    The provided `ignore_index` can be used as padding element in the `target_indices`
    field.

    Per definition, we have (https://en.wikipedia.org/wiki/Cross_entropy):
        CE(p,q) = -(p * log(q)).sum()
    With a provided weight per class, the computation becomes:
        CE(p,q,w) = -(p * log(q)).sum() * (p * w).sum()

    Parameters
    ----------
    pred: tensor
        a tensor of size `N x C x *` where N is the batch size, C is the number
        of classes, and `*` represents any other dimensions. This tensor represents
        the logit values.
    target_indices: tensor
        a tensor of size `N x D x *` where N is the batch size, D <= C is the number
        of classes present in the soft distribution, and `*` represents
        any other dimensions. It must match the tailing dimensions of `pred`.
    target_probas: tensor
        a tensor of same size as `target_indices` representing the probability
        associated to each class therein.
    weight: tensor
        a manual rescaling weight given to each class. It is a 1-D tensor of size
        `C`. Default: None
    reduction: str
        Specifies the reduction to apply to the output: 'none' | 'mean' | 'sum'.
        Default: 'mean'
    ignore_index: int
        Specifies a target value that is ignored and does not contribute
        to the gradient. Default: -1

    Return
    ----------
    result: tensor
        the computed loss.

    """
    target_indices = target_indices.long()
    assert reduction in ["none", "mean", "sum"]

    dim = pred.dim()
    if dim < 2:
        raise ValueError("Expected 2 or more dimensions (got {})".format(dim))

    dim = target_indices.dim()
    if dim < 2:
        raise ValueError("Expected 2 or more dimensions (got {})".format(dim))

    assert (weight is None) or (weight.dim() == 1 and weight.size(0) == pred.size(1))

    if pred.size(0) != target_indices.size(0):
        raise ValueError(
            f"Expected input batch_size ({pred.size(0)}) to match "
            f"target batch_size ({target_indices.size(0)})."
        )

    if pred.size(1) < target_indices.size(1):
        raise ValueError(
            f"Expected input class_size ({pred.size(1)}) to be greater/equal"
            f"than target class_size ({target_indices.size(1)})."
        )
    if target_indices.size()[2:] != pred.size()[2:]:
        out_size = target_indices.size()[:2] + pred.size()[2:]
        raise ValueError(
            f"Expected target_indices size {out_size} (got {target_indices.size()})"
        )
    if target_indices.size() != target_probas.size():
        raise ValueError(
            f"Expected target_probas size {target_indices.size()} "
            f"(got {target_probas.size()})"
        )

    log_probs = torch.nn.functional.log_softmax(pred, dim=1)
    mask = target_indices != ignore_index
    masked_indices = target_indices * mask
    tmp_weight = 1.0 if weight is None else weight[masked_indices]
    avg_log_probs = (mask * log_probs.gather(1, masked_indices) * target_probas).sum(
        dim=1
    )
    avg_weight = (
        1.0 if weight is None else (tmp_weight * mask * target_probas).sum(dim=1)
    )
    result = -(avg_weight * avg_log_probs)

    if reduction == "sum":
        result = result.sum()
    elif reduction == "mean":
        result = result.mean() if weight is None else result.sum() / avg_weight.sum()

    return result




您提供的代码是一个 Python 脚本，它定义了一个名为 soft_cross_entropy 的函数。此函数使用软标签计算交叉熵损失。软标签是通过提供目标指标及其相应的概率来定义的。 以下是代码不同部分的细分： 该脚本从 PyTorch 库导入各种模块，包括 torch、torch.nn.functional、torch.distributions 和 torch.nn。 该脚本将 CUDA 设备设置为 0 并定义一个小常量 eps。 soft_cross_entropy 函数有几个参数：pred、target_indices、target_probas、weight、reduction 和 ignore_index。 pred 表示 logit 值，预计是大小为 N x C x * 的张量，其中 N 是批量大小，C 是类数，* 表示任何其他维度。 target_indices 是大小为 N x D x * 的张量，其中 D 是软分布中存在的类数 (D <= C)，* 表示任何其他维度。它必须匹配 pred 的尾部尺寸。 target_probas 是一个与 target_indices 大小相同的张量，表示与 target_indices 中每个类别相关联的概率。 weight 是一个大小为 C 的可选张量，它为每个类提供手动重新调整权重。默认情况下，它设置为无。 reduction 指定应用于输出的缩减：“none”、“mean”或“sum”。默认值为“均值”。 ignore_index 是一个整数，指定要忽略且不影响梯度的目标值。默认值为 -1。 该函数对输入张量执行多项验证，如果维度或大小与预期值不匹配，则会引发错误。 在函数内部，使用 pred 上的 log softmax 函数计算对数概率。 创建一个掩码以从计算中排除忽略的索引。 通过收集与屏蔽索引对应的对数概率并将它们与目标概率相乘来计算损失。 如果提供权重张量，则损失会进一步加权。 最后，根据指定的减少方法（'none'、'mean' 或 'sum'）减少损失。 总体而言，此代码提供了支持软标签的交叉熵损失的自定义实现。

In [ ]:
class sym_acquire_func(nn.Module):
    """docstring for Net"""
    def __init__(self, state_size, action_size):
        super(sym_acquire_func, self).__init__()
        self.fc1 = nn.Linear(state_size, 1024*2)
        self.fc2 = nn.Linear(1024*2, 2048*1)
        self.fc3 = nn.Linear(2048*1, 1024*2)
        self.out = nn.Linear(1024*2, action_size)

    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        action_prob = F.softmax(self.out(x), dim = 1)

        return action_prob



您提供的代码使用 PyTorch nn.Module 类定义了一个名为 sym_acquire_func 的神经网络模型。该模型表示将输入状态向量映射到动作概率的函数逼近器。以下是代码不同组件的细分： sym_acquire_func 类继承自 nn.Module 类，它是 PyTorch 中所有神经网络模块的基类。 __init__ 方法是类的构造函数。它以 state_size 和 action_size 作为输入参数。 在构造函数内部，模型架构是使用完全连接的线性层 (nn.Linear) 定义的。 输入层 (self.fc1) 采用大小为 state_size 的状态向量并输出大小为 1024*2 的张量。 第二层 (self.fc2) 获取前一层 (1024*2) 的输出并生成大小为 2048*1 的张量。 第三层 (self.fc3) 获取前一层 (2048*1) 的输出并生成大小为 1024*2 的张量。 输出层 (self.out) 获取前一层 (1024*2) 的输出并产生一个大小为 action_size 的张量，它表示动作概率。 前向方法定义了神经网络的前向传递。 输入 x 通过每个线性层，然后是 ReLU 激活函数 (F.relu)。 最后一个线性层的输出通过一个softmax函数（F.softmax）得到动作概率（action_prob）。 动作概率作为前向传递的输出返回。 总的来说，这段代码定义了一个简单的神经网络模型，具有三个隐藏层和一个输出层，可用于根据输入状态向量获取动作概率。

In [ ]:
class diagnosis_func(nn.Module):
    """docstring for Net"""
    def __init__(self, state_size, disease_size):
        super(diagnosis_func, self).__init__()
        self.fc1 = nn.Linear(state_size, 1024*2)
        self.fc2 = nn.Linear(1024*2, 1024*2)
        self.out = nn.Linear(1024*2, disease_size)

    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        output = self.out(x)
        return output


您提供的代码定义了另一个名为 diagnosis_func 的神经网络模型，它也是 PyTorch 中 nn.Module 类的子类。该模型旨在根据输入状态向量诊断疾病。这是代码的细分： diagnosis_func 类继承自 nn.Module 类。 构造函数（__init__ 方法）将 state_size 和 disease_size 作为输入参数。 在构造函数内部，模型架构是使用完全连接的线性层 (nn.Linear) 定义的。 输入层 (self.fc1) 采用大小为 state_size 的状态向量并生成大小为 1024*2 的张量。 第二层 (self.fc2) 获取前一层 (1024*2) 的输出并生成大小为 1024*2 的张量。 输出层 (self.out) 获取前一层 (1024*2) 的输出并产生大小为 disease_size 的张量，代表诊断输出。 前向方法定义了神经网络的前向传递。 输入 x 通过每个线性层，然后是 ReLU 激活函数 (F.relu)。 最后一个线性层的输出作为诊断输出返回。 总之，diagnosis_func 模型获取输入状态向量，通过两个具有 ReLU 激活的隐藏层对其进行处理，并根据指定的 disease_size 生成诊断输出。该模型可用于疾病诊断任务。

In [ ]:
class Policy_Gradient_pair_model(object):
    def __init__(self, state_size, disease_size, symptom_size, LR = 1e-4, Gamma = 0.99, Eta = 0.01):

        self.policy = sym_acquire_func(state_size, symptom_size)
        self.classifier = diagnosis_func(state_size, disease_size)
        self.lr = LR
        self.policy.cuda(CUDA_device)
        self.classifier.cuda(CUDA_device)
        self.optimizer_p = torch.optim.Adam(self.policy.parameters(), lr=LR/5)
        self.optimizer_c = torch.optim.Adam(self.classifier.parameters(), lr=LR)
        self.counter = 1
        self.cross_entropy = nn.CrossEntropyLoss()
        # hyper_params
        self.gamma = Gamma
        self.eta = Eta

    def create_batch(self, states, rewards_s, action_s, true_d, true_diff_ind, true_diff_prob):
        
        cumulate_R_s = []
        R_s = 0
        for r_s in rewards_s[::-1]:
            R_s = r_s + self.gamma * R_s
            cumulate_R_s.insert(0, R_s)
        
        rewards_s = np.array(rewards_s)
        ave_rewards_s = np.mean(np.sum(rewards_s, axis = 0))

        cumulate_R_s = np.array(cumulate_R_s).T
        states = np.array(states).swapaxes(0, 1)
        action_s = np.array(action_s).T
        true_d = np.array(true_d).T
        true_diff_ind = (
            None if true_diff_ind[0] is None
            else np.array(true_diff_ind).swapaxes(0, 1)
        )
        true_diff_prob = (
            None if true_diff_prob[0] is None
            else np.array(true_diff_prob).swapaxes(0, 1)
        )

        valid_sample = (cumulate_R_s != 0)

        self.batch_rewards_s = torch.from_numpy(cumulate_R_s[valid_sample]).float()
        self.batch_states = torch.from_numpy(states[valid_sample]).float()
        self.batch_action_s = torch.from_numpy(action_s[valid_sample])
        self.batch_true_d = torch.from_numpy(true_d[valid_sample])
        self.batch_true_diff_ind = (
            None if true_diff_ind is None else torch.from_numpy(true_diff_ind[valid_sample])
        )
        self.batch_true_diff_prob = (
            None if true_diff_prob is None else torch.from_numpy(true_diff_prob[valid_sample])
        )

        return valid_sample, len(self.batch_rewards_s), ave_rewards_s
    
    @torch.no_grad() 
    def choose_action_s(self, state, deterministic=False):

        self.policy.eval()
        state = torch.from_numpy(state).float()
        probs = self.policy.forward(state.cuda(CUDA_device))
        m = Categorical(probs)
        if not deterministic:
            action = m.sample().detach().cpu().squeeze().numpy()
        else:
            action = torch.max(probs, dim = 1)[1].detach().cpu().squeeze().numpy()

        return action
    
    @torch.no_grad()
    def choose_diagnosis(self, state):

        self.classifier.eval()
        state = torch.from_numpy(state).float()
        output = self.classifier.forward(state.cuda(CUDA_device)).detach().cpu().squeeze()

        return torch.max(output, dim = 1)[1].numpy(), torch.softmax(output, dim = 1).numpy()
    
    def update_param_rl(self):  

        self.policy.train() 
        self.optimizer_p.zero_grad()
        state_tensor = self.batch_states.cuda(CUDA_device)
        reward_tensor = self.batch_rewards_s.cuda(CUDA_device)
        action_s_tensor = self.batch_action_s.cuda(CUDA_device)
        prob_tensor = self.policy.forward(state_tensor)
        #Policy Loss
        m = Categorical(prob_tensor)
        log_prob_tensor = m.log_prob(action_s_tensor)
        policy_loss = - (log_prob_tensor * (reward_tensor)).mean()
        #entropy Loss
        entropy_loss = - torch.max(torch.tensor([self.eta-self.counter*0.00001, 0])) * m.entropy().mean()
        loss = policy_loss + entropy_loss
        loss.backward()
        self.optimizer_p.step()

        self.counter += 1

    def update_param_c(self):

        self.classifier.train()
        self.optimizer_c.zero_grad()
        state_tensor = self.batch_states.cuda(CUDA_device)
        label_tensor = self.batch_true_d.cuda(CUDA_device)
        diff_ind_tensor = (
            None if self.batch_true_diff_ind is None
            else self.batch_true_diff_ind.cuda(CUDA_device)
        )
        diff_prob_tensor = (
            None if self.batch_true_diff_prob is None
            else self.batch_true_diff_prob.cuda(CUDA_device)
        )
        output_tensor = self.classifier.forward(state_tensor)
        if diff_ind_tensor is None or diff_prob_tensor is None:
            loss = self.cross_entropy(output_tensor, label_tensor)
        else:
            loss = soft_cross_entropy(output_tensor, diff_ind_tensor, diff_prob_tensor)
        loss.backward()
        self.optimizer_c.step()
    
    def change_lr(self):
        self.lr = self.lr /2 
        if self.lr < 1e-5:
            self.lr = 1e-4
        for param_group in self.optimizer_p.param_groups:
            param_group['lr'] = self.lr/2
        for param_group in self.optimizer_c.param_groups:
            param_group['lr'] = self.lr
        for param_group in self.optimizer_m.param_groups:
            param_group['lr'] = self.lr/2
  
    def save_model(self, args, prefix=""):
        info = str(args.dataset) + '_' + str(args.threshold) + '_' + str(args.mu) + '_' + str(args.nu) + '_' + str(args.trail)
        torch.save(self.policy.state_dict(), os.path.join(args.save_dir, f"{prefix}policy_{info}.pth"))
        torch.save(self.classifier.state_dict(), os.path.join(args.save_dir, f"{prefix}classifier_{info}.pth"))
       
    def load_model(self, args, prefix=""):
        info = str(args.dataset) + '_' + str(args.threshold) + '_' + str(args.mu) + '_' + str(args.nu) + '_' + str(args.trail)
        self.policy.load_state_dict(torch.load(os.path.join(args.checkpoint_dir, f"{prefix}policy_{info}.pth"), map_location='cuda:0'))
        self.classifier.load_state_dict(torch.load(os.path.join(args.checkpoint_dir, f"{prefix}classifier_{info}.pth"), map_location='cuda:0'))


    def train(self):
        self.policy.train()
        self.classifier.train()

    def eval(self):
        self.policy.eval()
        self.classifier.eval()
        


您共享的代码定义了一个名为 Policy_Gradient_pair_model 的类。 这个类实现了一个结合了策略网络（sym_acquire_func）和诊断网络（diagnosis_func）的策略梯度模型。 这是代码的细分：

构造函数（__init__ 方法）有几个参数：

state_size：输入状态向量的大小。
disease_size：诊断的疾病数量。
symptom_size：可能的症状数。
LR：优化的学习率（默认值：1e-4）。
Gamma：未来奖励的折扣因子（默认值：0.99）。
Eta：熵正则化权重（默认值：0.01）。
在构造函数内部，策略网络 (sym_acquire_func) 和诊断网络 (diagnosis_func) 被初始化。

使用 cuda(CUDA_device) 将策略和诊断网络移动到 GPU（如果可用）。

使用具有指定学习率的 Adam 优化器，为策略网络 (self.optimizer_p) 和诊断网络 (self.optimizer_c) 创建两个单独的优化器。

初始化模型的其他属性和超参数。

Policy_Gradient_pair_model 类也提供了几个方法：

create_batch：获取一批状态、奖励、动作、真实诊断和其他相关信息，并为训练准备数据。 返回有关批次的一些统计信息。
choose_action_s：给定状态，根据策略网络的输出概率选择要采取的动作（症状）。 该动作可以随机采样或确定性地选择。
choose_diagnosis：给定状态，使用诊断网络预测疾病诊断。
update_param_rl：使用策略梯度优化更新策略网络的参数。 根据所选动作的奖励和对数概率计算策略损失和熵损失。
update_param_c：使用交叉熵损失更新诊断网络的参数。 根据预测诊断和真实诊断计算损失。
change_lr：将优化器的学习率降低一半。
save_model 和 load_model：将模型参数保存到文件或从文件加载模型参数。
train and eval：将模型设置为训练或评估模式。
总体而言，该课程将强化学习与策略梯度优化以及监督学习与交叉熵损失相结合，以训练基于症状的疾病诊断模型。